In [1]:
#MultiClass classifier. Predicts one attribute as one of many labels rather than just 2 possible labels.
#This assumes there is only one right answer. We could do multilabel classification and present multiple city solutions to the user?
#https://scikit-learn.org/stable/modules/multiclass.html    a look at different kinds of models
# Import necessary libraries
#     ut.run_script(re.DB_WEATHER_DESC_SCRIPT)     copy this kind of thing to add mountains/water data

# method of going through multiple models
# https://medium.com/analytics-vidhya/testing-multiple-machine-learning-models-at-once-without-getting-a-headache-5aefb0e7df03    simple good
# https://github.com/j-planet/machine-learning-big-loop   more advanced, but better. (get parameters.py from this, and the cell for running multiple things


# Import necessary modules from repository
from model.data.pipeline import aggregate

In [2]:
df = aggregate.get_agg_data_frame()

df

,city,year,month,temperature,humidity,pressure,wind_speed,rain,snow,cloud,fog,extreme_weather
0,Albuquerque,2012,10,282.594630,40.870130,1017.681818,1.980519,0.006431,0.000000,0.456592,0.000000,0.000000
1,Albuquerque,2012,11,276.787692,44.012821,994.980769,1.801282,0.019802,0.001650,0.526403,0.003300,0.003300
2,Albuquerque,2012,12,271.423110,55.940476,1010.517857,1.821429,0.010401,0.001486,0.754829,0.001486,0.004458
3,Albuquerque,2013,01,268.405723,57.488372,999.837209,2.453488,0.036023,0.002882,0.576369,0.011527,0.001441
4,Albuquerque,2013,02,271.623359,54.910256,953.775641,2.275641,0.011024,0.007874,0.587402,0.001575,0.009449
...,...,...,...,...,...,...,...,...,...,...,...,...
1666,Seattle,2017,07,286.727283,84.059140,1020.053763,1.607527,0.020161,0.000000,0.442204,0.081989,0.000000
1667,Seattle,2017,08,287.951183,84.413978,1017.043011,1.553763,0.025538,0.000000,0.302419,0.290323,0.106183
1668,Seattle,2017,09,285.830661,87.100000,1015.833333,1.738889,0.181944,0.000000,0.465278,0.226389,0.043056
1669,Seattle,2017,10,280.271667,92.629032,1019.784946,2.048387,0.231183,0.000000,0.475806,0.314516,0.002688


In [3]:
labels = df["city"]
features = df.drop(['city','year'], axis = 1)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
featPrepped = scaler.fit_transform(features)

In [6]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, lab_train, lab_test = train_test_split(featPrepped, labels, random_state = 42)
# stratify by city for separating training and testing. stratified sampling

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(feat_train, lab_train)

clf.score(feat_test,lab_test)
# low scare shows we do not have enough destinctions between cities.
# Score got worse when we expanded the data by years. went from like 47 % to 35 %

0.3588516746411483

In [8]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(feat_train, lab_train)

clf.score(feat_test,lab_test)

0.2822966507177033

```
SELECT DISTINCT weather_description
FROM USWeather;

SELECT <columns>
FROM <table>
#WHERE <condition>
#ORDER BY <column> <asc/desc>;
```

In [9]:
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,  GradientBoostingClassifier


log_reg_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors":3}, {"n_neighbors":5}]
naive_bayes_params = [{}]
svc_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}]

modelclasses = [
    ["GradientBoostingClf", GradientBoostingClassifier()],
    ["ExtraTreeClf", ExtraTreesClassifier()],
    ["decision tree", DecisionTreeClassifier()],
    ["random forest", RandomForestClassifier()],
    ["k neighbors", KNeighborsClassifier()],
    ["NearestCentroid", NearestCentroid()],
    #["RadiusNeighbor", RadiusNeighborsClassifier()],
    ["GaussianProcess", GaussianProcessClassifier()],
    ["naive bayes Gaussian NB", GaussianNB()],
    ["naive bayes Bernoulli NB", BernoulliNB()],
    ["support vector machines", SVC()],
    ["Nu support vector machines", NuSVC()],
    ["NN MPL", MLPClassifier()]
]

insights = []
for modelname, model in modelclasses:
    print(modelname + "\n")
    model.fit(feat_train, lab_train)
    score = model.score(feat_test, lab_test)
    insights.append((modelname, model, score))


insights.sort(key=lambda x:x[-1], reverse=True)
for modelname, model, score in insights:
    print(modelname, score)





GradientBoostingClf

ExtraTreeClf

decision tree

random forest

k neighbors

NearestCentroid

GaussianProcess

naive bayes Gaussian NB

naive bayes Bernoulli NB

support vector machines

Nu support vector machines

NN MPL

NN MPL 0.4019138755980861
Nu support vector machines 0.38516746411483255
ExtraTreeClf 0.35406698564593303
random forest 0.35406698564593303
GradientBoostingClf 0.35167464114832536
support vector machines 0.3492822966507177
GaussianProcess 0.3397129186602871
k neighbors 0.29904306220095694
NearestCentroid 0.2727272727272727
decision tree 0.22248803827751196
naive bayes Bernoulli NB 0.21770334928229665
naive bayes Gaussian NB 0.19138755980861244


c:\users\smahe\ai\ml-team3\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
